In [ ]:
import pickle
import pandas as pd
import numpy as np 
import itertools
import keras
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img 
from keras.models import Sequential 
from keras import optimizers
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense,Conv2D, MaxPool2D
from keras import applications  
from keras.utils.np_utils import to_categorical  
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
%matplotlib inline
import math  

In [ ]:
pickle_in = open("train_images.pickle","rb") 
train_images = pickle.load(pickle_in)

In [ ]:
pickle_in = open("train_labels.pickle","rb")
train_labels = pickle.load(pickle_in)

In [ ]:
pickle_in = open("validation_images.pickle","rb")
validation_images = pickle.load(pickle_in)

In [ ]:
pickle_in = open("validation_labels.pickle","rb")
validation_labels = pickle.load(pickle_in)

In [ ]:
pickle_in = open("test_images.pickle","rb")
test_images = pickle.load(pickle_in)

In [ ]:
pickle_in = open("test_labels.pickle","rb")
test_labels = pickle.load(pickle_in)

In [ ]:
print(train_images.shape)
print(validation_images.shape)
print(test_images.shape)

In [ ]:
train_labels = to_categorical(train_labels)
print(train_labels.shape)

In [ ]:
validation_labels = to_categorical(validation_labels)
print(validation_labels.shape)

In [ ]:
vgg19 = keras.applications.vgg19
conv_model = vgg19.VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3))
x = keras.layers.Flatten()(conv_model.output)
x = keras.layers.Dense(100,activation=keras.layers.LeakyReLU(alpha=0.3))(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(50,activation=keras.layers.LeakyReLU(alpha=0.3))(x)
x = keras.layers.Dropout(0.3)(x)
predictions = keras.layers.Dense(3,activation='softmax')(x)
full_model = keras.models.Model(inputs=conv_model.input,outputs=predictions)

In [ ]:
full_model.summary()

In [ ]:
for layer in conv_model.layers:
    layer.trainable = False

In [ ]:
full_model.summary()

In [ ]:
full_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=1e-4),
              metrics=['acc'])  

In [ ]:
history = full_model.fit(train_images,train_labels,batch_size=32,epochs=7,validation_data=(validation_images,validation_labels))

In [ ]:
full_model.evaluate(test_images,test_labels)

In [ ]:
full_model.save('vgg19.h5')

In [ ]:
from keras.models import load_model

In [ ]:
model = load_model('vgg19.h5',custom_objects={'LeakyReLU':keras.layers.LeakyReLU})

In [ ]:
def prepare(filepath):
    IMG_SIZE = 224  # 50 in txt-based
    img_array = cv2.imread(filepath, cv2.IMREAD_COLOR)  # read in the image, convert to grayscale
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

In [ ]:
prediction = np.argmax(model.predict([prepare('Figure.jpg')]),axis=-1)

In [ ]:
CATEGORIES = ["Basket", "Coin", "Figure"]

In [ ]:
print(CATEGORIES[int(prediction)])

In [ ]:
print('test images', test_images)
preds = np.round(model.predict(test_images),0) 
#to fit them into classification metrics and confusion metrics, some additional modificaitions are required
print('rounded test_labels', preds)

In [ ]:
animals = ['Basket', 'Coin', 'Figure']
classification_metrics = metrics.classification_report(test_labels, preds, target_names=animals )
print(classification_metrics)

In [ ]:
categorical_test_labels = to_categorical(categorical_test_labels)
categorical_preds = to_categorical(categorical_preds)

In [ ]:
categorical_test_labels = pd.DataFrame(test_labels).idxmax(axis=1)
categorical_preds = pd.DataFrame(preds).idxmax(axis=1)

In [ ]:
confusion_matrix= confusion_matrix(categorical_test_labels, categorical_preds)

In [ ]:
def plot_confusion_matrix(cm, classes,
             normalize=False,
             title='Confusion matrix',
             cmap=plt.cm.Blues):
    #Add Normalization Option
    '''prints pretty confusion metric with normalization option '''
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    
#     print(cm)
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
    
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
plot_confusion_matrix(confusion_matrix, ['Basket', 'Coin', 'Figure'],normalize=True)